In [36]:
spark.version

u'2.2.1'

In [37]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

wget: /usr/local/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /usr/local/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /usr/local/lib/libssl.so.1.0.0: no version information available (required by wget)
--2018-07-16 08:04:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [text/plain]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2018-07-16 08:04:34 (84.5 MB/s) - ‘iris.data’ saved [4551/4551]



In [ ]:
! hdfs dfs -put iris.data /

In [38]:
! hdfs dfs -ls /

18/07/16 08:04:42 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: 1.6.7-hadoop2
Found 4 items
-rw-r--r--   2 root   hadoop       4551 2018-07-16 07:06 /iris.data
drwxrwxrwt   - mapred hadoop          0 2018-07-16 06:34 /mapred
drwxrwxrwt   - mapred hadoop          0 2018-07-16 06:35 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2018-07-16 07:05 /user


In [ ]:
# Import Libaries
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *

In [39]:
# get into DataFrame
csvFile = spark.read.csv('/iris.data', inferSchema=True)
diz = {"Iris-setosa":"1", "Iris-versicolor":"2", "Iris-virginica":"3" }
df = csvFile.na.replace(diz,1,"_c4")
df2 = df.withColumn("label",df["_c4"].cast(IntegerType())) \
  .withColumnRenamed("_c0","sepal_length") \
  .withColumnRenamed("_c1","sepal_width") \
  .withColumnRenamed("_c2","petal_length") \
  .withColumnRenamed("_c3","petal_width") 
train,test = df2.randomSplit([0.75,0.25])

In [29]:
df2.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- _c4: string (nullable = true)
 |-- label: integer (nullable = true)



In [30]:
df2.show()

+------------+-----------+------------+-----------+---+-----+
|sepal_length|sepal_width|petal_length|petal_width|_c4|label|
+------------+-----------+------------+-----------+---+-----+
|         5.1|        3.5|         1.4|        0.2|  1|    1|
|         4.9|        3.0|         1.4|        0.2|  1|    1|
|         4.7|        3.2|         1.3|        0.2|  1|    1|
|         4.6|        3.1|         1.5|        0.2|  1|    1|
|         5.0|        3.6|         1.4|        0.2|  1|    1|
|         5.4|        3.9|         1.7|        0.4|  1|    1|
|         4.6|        3.4|         1.4|        0.3|  1|    1|
|         5.0|        3.4|         1.5|        0.2|  1|    1|
|         4.4|        2.9|         1.4|        0.2|  1|    1|
|         4.9|        3.1|         1.5|        0.1|  1|    1|
|         5.4|        3.7|         1.5|        0.2|  1|    1|
|         4.8|        3.4|         1.6|        0.2|  1|    1|
|         4.8|        3.0|         1.4|        0.1|  1|    1|
|       

In [31]:
# Model
assembler = VectorAssembler(
    inputCols=["sepal_length","sepal_width","petal_length","petal_width"],
    outputCol="features")
lr = LogisticRegression(labelCol="label", predictionCol="prediction")
paramGrid = ParamGridBuilder().build()

#Pipeline
pipeline = Pipeline(stages=[assembler, lr])

In [32]:
model = pipeline.fit(train)
predictions = model.transform(train)

In [33]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label")

In [34]:
evaluator.evaluate(predictions)

1.0

In [35]:
evaluator.evaluate(model.transform(test))

0.9114149008885851